In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="TeieUHWBZSJGpBxZWh10")
project = rf.workspace("mishmeria").project("failure-pallet-detection-94gji")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [5]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

In [2]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU in use:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


PyTorch version: 2.5.1+cu121
CUDA available: True
GPU in use: NVIDIA GeForce RTX 4070 Laptop GPU


In [7]:
model.train(
    data="failure-pallet-detection-2/data.yaml",
    epochs=100,          # enough to converge on 3.8k images
    imgsz=640,           # matches Roboflow preprocessing
    batch=16,            # adjust if OOM, try 8 if needed
    patience=20,         # early stopping if no improvement
    optimizer="AdamW",   # better on small/medium datasets
    device=0,            # force CUDA GPU
    workers=2,           # safe for Jupyter; increase if CPU is free
    project="runs/detect", name="pallet_defect_v1"
)

New https://pypi.org/project/ultralytics/8.3.189 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.183  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=failure-pallet-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pallet_defect_v1, nbs=6

train: Scanning D:\ClassifyModel\Failure-Pallet-Detection-2\train\labels... 2682 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2682/2682 [00:05<00:00, 492.54it/s]


train: New cache created: D:\ClassifyModel\Failure-Pallet-Detection-2\train\labels.cache
val: Fast image access  (ping: 0.10.0 ms, read: 8.51.1 MB/s, size: 50.9 KB)


val: Scanning D:\ClassifyModel\Failure-Pallet-Detection-2\valid\labels... 276 images, 0 backgrounds, 0 corrupt: 100%|██████████| 276/276 [00:00<00:00, 456.20it/s]

val: New cache created: D:\ClassifyModel\Failure-Pallet-Detection-2\valid\labels.cache


...

                   all        276        452      0.859      0.887       0.88      0.682
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 47, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



67 epochs completed in 0.419 hours.
Optimizer stripped from runs\detect\pallet_defect_v1\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\pallet_defect_v1\weights\best.pt, 6.3MB

Validating runs\detect\pallet_defect_v1\weights\best.pt...
Ultralytics 8.3.183  Python-3.11.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  4.64it/s]


                   all        276        452      0.866      0.867      0.875      0.689
               Cracked        122        175      0.735      0.737      0.754      0.413
                Pallet        276        277      0.998      0.996      0.995      0.966
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs\detect\pallet_defect_v1


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001FF3CB4C5D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

In [ ]:
import cv2
from ultralytics import YOLO
model = YOLO("D:/ClassifyModel/runs/detect/pallet_defect_v1/weights/best.pt")
cap = cv2.VideoCapture("C:/Program Files (x86)/HIK IN/HIK IN/storage/2025-08-26 14.34.41.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.25)
    annotated_frame = results[0].plot()

    cv2.imshow("YOLO Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # press q to quit
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 Cracked, 1 Pallet, 37.5ms
Speed: 5.9ms preprocess, 37.5ms inference, 8.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Cracked, 1 Pallet, 5.9ms
Speed: 1.2ms preprocess, 5.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Cracked, 1 Pallet, 5.9ms
Speed: 1.5ms preprocess, 5.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Cracked, 1 Pallet, 5.6ms
Speed: 1.3ms preprocess, 5.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Cracked, 1 Pallet, 7.6ms
Speed: 1.4ms preprocess, 7.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Cracked, 1 Pallet, 7.1ms
Speed: 1.3ms preprocess, 7.1ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Cracked, 1 Pallet, 6.5ms
Speed: 1.9ms preprocess, 6.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Cracked, 1 Pallet, 6.0ms
Speed: 1.5ms p

KeyboardInterrupt: 

: 